In [64]:
import ipywidgets as ipw

from elasticsearch import Elasticsearch
from typing import Dict, Any, Tuple, List
from ipyevents import Event
from functools import partial

es = Elasticsearch()


def create_image_carousel(image_data: Dict[str, str]) -> Tuple[ipw.HBox, List[Event]]:
    carousel_child = []
    ev_handles = []
    for img in image_data:
        img_box = ipw.Image.from_url(img['url'])
        desc = img['description']
        im_event = Event()
        im_event.source = img_box
        im_event.watched_events = ['click']
        im_event.on_dom_event(partial(on_img_clicked, data=img))
        ev_handles.append(im_event)
        carousel_child.append(img_box)

    return ipw.HBox(carousel_child), ev_handles


def perform_painting_search(index: str, query: str):
    query = {
        "size": 10,
        "query": {
            "match": {
                "description": query
            }
        },
        "aggs": {
            "authors": {
                "terms": {
                    "field": "author"
                }
            },
            "genres": {
                "terms": {
                    "field": "genre"
                }
            },
            "Total authors": {
                "cardinality": {
                    "field": "author"
                }
            },
            "Styles": {
                "cardinality": {
                    "field": "style"
                }
            }
        }

    }
    res = es.search(index=index, body=query)
    img_data = [x['_source'] for x in res['hits']['hits']]
    return img_data, res['aggregations']


description_box = ipw.HTML(value="")
aggregation_stats = ipw.HTML(value="")


def on_img_clicked(event, data={}):
    desc = data.get("description", "")
    author = data.get("author", "")
    title = data.get("title", "")
    html_val = f"""
    <div> 
        <h2>Author: {author}</h2>
        <h2>Title: {title}</h2>
        <p style="font-family:sans-serif">{desc}</p>
    </div>
    """
    description_box.value = html_val


def fill_stats(aggregations: Dict[str, Any]):
    authors = aggregations.get('Total authors', "")
    if authors:
        authors = authors['value']
    styles = aggregations.get('Styles', "")
    if styles:
        styles = styles['value']

    def gen_list(input_doc):
        lst = "\n".join(
            [f"<li>{el['key']}: {el['doc_count']}</li>" for el in input_doc])
        return f"<ul>{lst}</ul>"

    lst = gen_list(aggregations['genres']['buckets'])
    html_val = f"""
    <div>
        <h3>#Different authors: {authors}</h3>
        <h3>#Different styles: {styles}</h3>
        <h3>Genres:</h3> 
        {lst}
    </div>
    """
    aggregation_stats.value = html_val


search_area = ipw.Text(description='Search:', placeholder="Enter a phrase")
search_button = ipw.Button(icon='fa-search')
search_box = ipw.HBox([search_area, search_button])

handles = []
output = ipw.Output()


def on_search_clicked(_):
    # with output:
    # clear out the previous search results
    handles = []
    aggregation_stats.value = ""
    description_box.value = ""

    query_text = search_area.value
    img_data, aggs = perform_painting_search(
        index="paintings", query=query_text)
    img_carousel_, ev_handles = create_image_carousel(
        img_data)
    img_carousel.children = img_carousel_.children
    handles = ev_handles
    fill_stats(aggs)


img_carousel = ipw.HBox()
search_button.on_click(on_search_clicked)
search_ = ipw.VBox([search_box, output, description_box,
                    img_carousel, aggregation_stats])
search_


In [50]:
es.indices.get_mapping( 'paintings')

/opt/homebrew/Caskroom/miniforge/base/envs/.art/lib/python3.9/site-packages/elasticsearch/connection/base.py:208: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.13/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


{'paintings': {'mappings': {'properties': {'author': {'type': 'keyword'},
    'description': {'type': 'text'},
    'genre': {'type': 'keyword'},
    'material': {'type': 'keyword'},
    'style': {'type': 'keyword'},
    'tags': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
    'title': {'type': 'text'},
    'url': {'type': 'text'}}}}}